In [1]:
from dataset import *
from train import *

In [32]:
import os
import pandas as pd

import collections

import sys

import torch
import torch.optim as optim

In [33]:
# Set parameters
EuroSat_Type = 'ALL'    

In [34]:
args = str(sys.argv)
target_task = args[1]
algorithm = args[2]
algorithm = "bandit"


In [44]:
target_task = "France"
target_size = 160

In [45]:
from pathlib import Path
output_path = Path("derived_data")
output_path.mkdir(parents = True, exist_ok = True)

In [46]:
if EuroSat_Type == 'RGB':
  data_folder = '/content/sample_data/'
  #root = os.path.join(data_folder, '2750/')
  root = '2750/'
  download_ON = os.path.exists(root)

  if not download_ON:
    # This can be long...
    #os.chdir(data_folder)
    os.system('wget http://madm.dfki.de/files/sentinel/EuroSAT.zip') #Just RGB Bands
    !unzip EuroSAT.zip
    download_ON = True
elif EuroSat_Type == 'ALL':
    root = 'ds/images/remote_sensing/otherDatasets/sentinel_2/tif/'
    download_ON = os.path.exists(root)
    if not download_ON:
      os.system('wget http://madm.dfki.de/files/sentinel/EuroSATallBands.zip') #All bands
      !unzip EuroSATallBands.zip
      download_ON = True

In [47]:
geo_df = pd.read_csv("metadata.csv")

In [48]:
data = torchvision.datasets.DatasetFolder(root=root,loader = iloader, transform = None, extensions = 'tif')
labels = [v[1] for (i, v) in enumerate(data)]

In [62]:
def prepare_input_data(geo_df, target_task, labels = None, 
                       train_size = .6, test_size = .5, target_size = 1600):
    
    geo_dict = geo_df.to_dict()
    countries = list(set(geo_dict["country"].values()))
    countries = [x for x in countries if str(x) != "nan"]
    id_countries = dict.fromkeys(countries)
    for k in id_countries.keys():
        id_countries[k] = [v for (i, v) in enumerate(geo_dict["id"]) if geo_dict["country"][i] == k]

    # create a dictionary for input data
    
    input_data = {
        "source_task": list(set(id_countries.keys()) - set(target_task)),
        "target_task": target_task
    }

    
    # all data, both source and target
    
    input_data["data_dict"] = {}
    for k in geo_dict.keys():
        input_data["data_dict"][k] = [geo_dict[k][i] for (i, v) in enumerate(geo_dict["country"].values()) if str(v) != "nan"]


        
    # split indices for source and target
    
    input_data["idx_source"] = [i for (i, v) in enumerate(input_data["data_dict"]['country']) if v != input_data["target_task"]]
    input_data["idx_target"] = [i for (i, v) in enumerate(input_data["data_dict"]['country']) if v == input_data["target_task"]]

    target_labels = list(set([labels[i] for i in input_data["idx_target"]]))

    
    # For source data, create a dictionary to record the countries
    
    input_data["source_dict"] = {}
    for k in geo_dict.keys():
        input_data["source_dict"][k] = [input_data["data_dict"][k][i] for i in input_data["idx_source"] if labels[i] in target_labels]

   
    # resample the target to make the number of samples is fixed
    
    if len(input_data["idx_target"]) >= target_size:
        input_data["idx_target"] = random.sample(input_data["idx_target"], k = target_size)
    else:
        input_data["idx_target"] = random.choices(input_data["idx_target"], k = target_size)
        
    
    # split the target data into train / validation / test sets
    
    y_target = [labels[i] for i in input_data["idx_target"]]
    input_data["idx_train"], idx_rest, _, y_rest = train_test_split(input_data["idx_target"],
                                                              y_target,
                                                              test_size = 1 - train_size,
                                                              random_state = 0, shuffle = True)
    input_data["idx_val"], input_data["idx_test"], _, _ = train_test_split(idx_rest,
                                                              y_rest,
                                                              test_size = test_size,
                                                              random_state = 0, shuffle = True)

    return input_data



In [63]:
input_data = prepare_input_data(geo_df, target_task, labels = labels,
                               target_size = target_size)

In [64]:
np.random.seed(0)
torch.cuda.manual_seed(0)
random.seed(0)

In [65]:
# prepare data ---

target_val_loader =  torch.utils.data.DataLoader(torch.utils.data.Subset(data, input_data["idx_val"]), 
                                              batch_size = 16, shuffle = True, num_workers = 0)
target_train_loader =  torch.utils.data.DataLoader(torch.utils.data.Subset(data, input_data["idx_train"]), 
                                                  batch_size = 16, shuffle = True, num_workers = 0)
target_test_loader =  torch.utils.data.DataLoader(torch.utils.data.Subset(data, input_data["idx_test"]), 
                                                  batch_size = 16, shuffle = True, num_workers = 0)



# initialize hyperparameters ---

bandit_selects = [None]
alpha = dict.fromkeys(input_data["source_task"], [1])
beta = dict.fromkeys(input_data["source_task"], [1])
pi = dict.fromkeys(input_data["source_task"], [0])


In [66]:
_, bandit_selects, accs, alpha, beta, pi = bandit_selection(data, input_data, 
                                                            n_epochs = 1, n_it = 2,
                                                            algorithm = algorithm, iter_samples = 160,
                                                           output_path = output_path)

/Users/miaoxinran/Desktop/source_selection/experiment_eurosat/dataset.py:92: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-z))


At iteration  1 , source country is  België / Belgique / Belgien , acc is  0.28125
